In [ ]:
## Load all packages
import os
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import numpy as np

In [ ]:
# Define paths
base_dir = "your-path-BA-data"   # Directory containing folders 2001, 2002, etc.
output_dir = "your-output-path"  # Directory to save the clipped rasters
portugal_shapefile = "path-and-data-Portugal-shapefile"  # Path to Portugal shapefile


In [ ]:
# Load the Portugal shapefile
portugal_shape = gpd.read_file(portugal_shapefile)
portugal_geom = [portugal_shape.geometry.unary_union]  # Get the unified geometry for clipping


In [ ]:
#  Loop through each year directory and clip to PT shape
for year in range(2001, 2021):
    year_dir = os.path.join(base_dir, str(year))
    
    if os.path.isdir(year_dir):
        # Ensure year-specific output directory exists
        year_output_dir = os.path.join(output_dir, str(year))
        os.makedirs(year_output_dir, exist_ok=True)
        
        # Process each file in the directory
        for filename in os.listdir(year_dir):
            if filename.endswith((".tif")) and any(suffix in filename for suffix in ["LC", "JD", "CL"]):
                file_path = os.path.join(year_dir, filename)
                
                # Open the raster file
                with rasterio.open(file_path) as src:
                    # Clip the raster with Portugal's geometry
                    clipped_image, clipped_transform = mask(src, portugal_geom, crop=True)
                    
                    # Update metadata with new transform and bounds
                    out_meta = src.meta.copy()
                    out_meta.update({
                        "driver": "GTiff",
                        "height": clipped_image.shape[1],
                        "width": clipped_image.shape[2],
                        "transform": clipped_transform
                    })
                    
                    # Define output file path with modified filename, saving to the year-specific folder
                    output_filename = f"{filename.replace('.tif', '')}_PT_shape.tif"
                    output_path = os.path.join(year_output_dir, output_filename)
                    
                    # Save the clipped raster
                    with rasterio.open(output_path, "w", **out_meta) as dest:
                        dest.write(clipped_image)
                
                print(f"Saved clipped file: {output_path}")